# Optimizing location of Ground Stations

Setting a specific continent/region, (we can also run cases without setting a continent, just anywhere in the world) complete nelder mead and find the best location for a ground station. Currently needs a bit more debugging to ensure convergence, and to speed up the solver in general.


## Setup Imports

(not relevant to repo, can skip) 

Adding module path to run correctly in examples folder

In [2]:
import sys
import os

# Add the path to the folder containing the module
module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/Users/gracekim/Documents/School_Everything_and_LEARNING/Stanford/SISL/loc-gsopt/src


## Imports

In [3]:
from sat_gen import satellites_from_constellation
from station_gen import gs_json, rand_gs_on_land,return_bdm_gs
from utils import load_earth_data, compute_all_gaps_contacts, compute_earth_interior_angle
from optimizations.nelder_mead import nelder_mead

import numpy as np

# Brahe Imports
import brahe as bh
import brahe.data_models as bdm
import brahe.access.access as ba

# random points function
import random_land_points as rlp

# Plotting Imports
import shapely
import cartopy.crs as ccrs
import cartopy.geodesic
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML # disply inline for ipynb 

## Setup for Earth Inertial Data and Scenario Generation

In [4]:
############################### SETUP: ###############################

# Make sure to load in earth inertial data every start time!
load_earth_data('data/iau2000A_finals_ab.txt')

############################### STEP 1: Satellites ###############################

# Loading in a satellite constellation 
CONSTELLATIONS = sorted(['YAM', 'UMBRA', 'SKYSAT', 'ICEYE', 'FLOCK', 'HAWK', 'CAPELLA', 'LEGION', 'WORLDVIEW', 'GEOEYE',
                  'NUSAT'])

constellation = CONSTELLATIONS[3]

satellites = satellites_from_constellation(constellation)

############################### STEP 2: Scenario Generation ###############################

# Setting up Epochs
epc0 = bh.Epoch(2024, 5, 20, 0, 0, 0) # This is the epoch of the orbital elements
epc10 = epc0 + 86400 # TODO: make this longer time period

continent = 'North America'  #'Antarctica'


Loading the latest Earth Orientation Data


## Nelder Mead

In [5]:

points = []

# random locations
# for i in range(3):
#     point = rlp.random_points(continent)[0] # Get a random point on land in Antartica
#     points.append((point[0],point[1]))

# Just put the random points here for now, maybe we do this for every continent?
points = [[ 179.4515625, 51.37260742],
 [ -61.17426758, 10.07802734],
 [-178.19453125, 51.88222656]]


# Cost function, we should be able tochange this if needed?
def cost_func_gap(new_gs, gs_list = [], satellites=satellites[0:10], epc_start = epc0, epc_end = epc10, plot = False):    
    gs_list.append(return_bdm_gs(new_gs[0], new_gs[1]))
    _, _, gaps_seconds = compute_all_gaps_contacts(satellites, gs_list ,epc_start, epc_end, plot)
    return np.mean(gaps_seconds)

print(points)
plot_points = nelder_mead(points, cost_func_gap, 5, continent)
print(plot_points)

[[179.4515625, 51.37260742], [-61.17426758, 10.07802734], [-178.19453125, 51.88222656]]
ITERATION:  0


(0.6285156249999915, 51.62741699)
REFLECTION
(0.6285156249999915, 51.62741699)
[-61.17426758, 10.07802734]
lat > 90
[(57.70594681616782, 90)]
(0.6285156249999915, 51.62741699)
[-61.17426758, 10.07802734]
[(57.70594681616782, 90), (29.167231220583904, 70.813708495)]
(0.6285156249999915, 51.62741699)
[-61.17426758, 10.07802734]
[(57.70594681616782, 90), (29.167231220583904, 70.813708495), (14.897873422791948, 61.2205627425)]
(0.6285156249999915, 51.62741699)
[-61.17426758, 10.07802734]
[(57.70594681616782, 90), (29.167231220583904, 70.813708495), (14.897873422791948, 61.2205627425), (7.76319452389597, 56.42398986625)]
(0.6285156249999915, 51.62741699)
[-61.17426758, 10.07802734]
[(57.70594681616782, 90), (29.167231220583904, 70.813708495), (14.897873422791948, 61.2205627425), (7.76319452389597, 56.42398986625), (4.19585507444798, 54.025703428125)]
(0.6285156249999915, 51.62741699)
[-61.17426758, 10.07802734]
[(57.70594681616782, 90), (29.167231220583904, 70.813708495), (14.89787342279194

In [6]:



############################### STEP 4: Plotting ###############################

# Now we can plot what a satellite can see from a given altitude
# Let's say we want to see what a satellite at 525 km can see if all observers
# are looking at it with at least 20 degrees elevation angle. This is an appropriate
# Value for a communications (Starlink/Kuiper) user terminal. If you wanted to get
# the maximum possible coverage limited by the Earth's curvature, you would use 0 degrees.

alt = 570 # Altitude in km
elevation_min = 20.0
lam = compute_earth_interior_angle(ele=elevation_min, alt=alt)


def animate(i):
    fig.clear()
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_global()
    ax.stock_img()
    points = plot_points[i]
    print(i)
    print(points)

    ax.set_title("Animation Counter Optimization: "+ str(i))
    ax.scatter(*zip(*points))
    for point in points:
        # Get a bunch of points in a circle space at the the right angle offset from the sub-satellite point to draw the view cone
        circle_points = cartopy.geodesic.Geodesic().circle(lon=point[0], lat=point[1], radius=lam*bh.R_EARTH, n_samples=100, endpoint=False)
        geom = shapely.geometry.Polygon(circle_points)
        ax.add_geometries((geom,), crs=ccrs.Geodetic(), alpha=0.5, edgecolor='none', linewidth=0)

    ax.set_yticks(np.arange(-90, 90, 30))
    ax.set_xticks(np.arange(-180, 180, 30))
    # sc = ax.scatter(x=longs, y=lats, c=gaps,cmap = 'cool')
    # plt.colorbar(sc)
    plt.grid()
    # plt.show()


print(len(plot_points))

fig = plt.figure()
ani = FuncAnimation(fig, animate, frames = range(len(plot_points)),repeat = False,interval =500)

HTML(ani.to_html5_video())
# plt.ylim(-1000,1000)
# plt.xlim(-1000,1000)
# plt.show()
# ani.save(filename="example.gif", writer="ffmpeg")


6
0
[[179.4515625, 51.37260742], [-61.17426758, 10.07802734], [-178.19453125, 51.88222656]]
0
[[179.4515625, 51.37260742], [-61.17426758, 10.07802734], [-178.19453125, 51.88222656]]


/Users/gracekim/.pyenv/versions/3.11.9/envs/gsopt/lib/python3.11/site-packages/cartopy/mpl/geoaxes.py:790: UserWarning: Approximating coordinate system <Geographic 2D CRS: +proj=lonlat +datum=WGS84 +ellps=WGS84 +no_defs +t ...>
Name: unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
 with the PlateCarree projection.
  warnings.warn(f'Approximating coordinate system {crs!r} with '
/Users/gracekim/.pyenv/versions/3.11.9/envs/gsopt/lib/python3.11/site-packages/cartopy/mpl/geoaxes.py:790: UserWarning: Approximating coordinate system <Geographic 2D CRS: +proj=lonlat +datum=WGS84 +ellps=WGS84 +no_defs +t ...>
Name: unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
 with the PlateCarree pro

1
[[179.4515625, 51.37260742], [-178.19453125, 51.88222656], (-138.65768704336418, 61.84054813976687)]
2
[(-138.65768704336418, 61.84054813976687), [-178.19453125, 51.88222656], (-158.59465516897365, 56.86412535864236)]
3
[(-158.59465516897365, 56.86412535864236), (-138.65768704336418, 61.84054813976687), (-119.05781096233778, 66.82244693840923)]


/Users/gracekim/.pyenv/versions/3.11.9/envs/gsopt/lib/python3.11/site-packages/cartopy/mpl/geoaxes.py:790: UserWarning: Approximating coordinate system <Geographic 2D CRS: +proj=lonlat +datum=WGS84 +ellps=WGS84 +no_defs +t ...>
Name: unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
 with the PlateCarree projection.
  warnings.warn(f'Approximating coordinate system {crs!r} with '
/Users/gracekim/.pyenv/versions/3.11.9/envs/gsopt/lib/python3.11/site-packages/cartopy/mpl/geoaxes.py:790: UserWarning: Approximating coordinate system <Geographic 2D CRS: +proj=lonlat +datum=WGS84 +ellps=WGS84 +no_defs +t ...>
Name: unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
 with the PlateCarree pro

4
[(-119.05781096233778, 66.82244693840923), (-138.65768704336418, 61.84054813976687), (-99.12084283672834, 71.79886971953374)]
5
[(-99.12084283672834, 71.79886971953374), (-138.65768704336418, 61.84054813976687), (-118.72071891775474, 66.81697092089138)]
